In [1]:
# library
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torchmetrics import R2Score, MeanSquaredError

r2score = R2Score()
msescore = MeanSquaredError()

torch.manual_seed(2)
np.random.seed(2)
torch.set_printoptions(precision=8)

In [2]:
# Sigma function for conditioning (u1 --> y1, u2 --> y2, ...)
def sigma(t, k, n):                             # t is the input, k is the size of one control, n is the window length
    a = np.array([])
    for i in range(n):
        for j in range(k):
            a = np.append(a, i+1)
    a = torch.tensor(a, dtype=torch.float32)
    alpha = 16
    return 1 - torch.sigmoid(alpha*(a-t-1.5))

In [3]:
# Model
class Net(torch.nn.Module):
  def __init__(self, n=4, p=8, noi=1, b0_size = 8, bi_size = 2, trunk_size = 8):
    super(Net, self).__init__()
    self.n   = n                    # horizon window length
    self.p   = p                    # size of branch and trunk output
    self.noi = noi                  # number of input
    self.k   = int(self.p/self.n)   # size of each sub-branch

    self.b0_size    = b0_size
    self.bi_size    = bi_size
    self.trunk_size = trunk_size
    
    # Branch x0
    self.input_x0  = torch.nn.Linear(1, self.b0_size)
    self.hidden_x0 = torch.nn.Linear(self.b0_size, self.b0_size)
    self.output_x0 = torch.nn.Linear(self.b0_size, self.p)

    # Branch 1 u
    self.input1_u  = torch.nn.Linear(self.noi, self.bi_size)
    self.hidden1_u = torch.nn.Linear(self.bi_size, self.bi_size)
    self.output1_u = torch.nn.Linear(self.bi_size, self.k)

    # Branch 2 u
    self.input2_u  = torch.nn.Linear(self.noi, self.bi_size)
    self.hidden2_u = torch.nn.Linear(self.bi_size, self.bi_size)
    self.output2_u = torch.nn.Linear(self.bi_size, self.k)

    # Branch 3 u
    self.input3_u  = torch.nn.Linear(self.noi, self.bi_size)
    self.hidden3_u = torch.nn.Linear(self.bi_size, self.bi_size)
    self.output3_u = torch.nn.Linear(self.bi_size, self.k)

    # Branch 4 u
    self.input4_u  = torch.nn.Linear(self.noi, self.bi_size)
    self.hidden4_u = torch.nn.Linear(self.bi_size, self.bi_size)
    self.output4_u = torch.nn.Linear(self.bi_size, self.k)

    # Trunk
    self.input_t   = torch.nn.Linear(1, self.trunk_size)
    self.hidden_t  = torch.nn.Linear(self.trunk_size, self.trunk_size)
    self.output_t  = torch.nn.Linear(self.trunk_size, self.p)

  def forward(self, x0, u, t):
    # h
    h = torch.selu(self.input_x0(x0))
    h = torch.selu(self.hidden_x0(h))
    h = self.output_x0(h)

    # f
    f1 = torch.selu(self.input1_u(u[:,0*self.noi:1*self.noi].reshape(-1,self.noi)))
    f1 = torch.selu(self.hidden1_u(f1))
    f1 = self.output1_u(f1)

    f2 = torch.selu(self.input2_u(u[:,1*self.noi:2*self.noi].reshape(-1,self.noi)))
    f2 = torch.selu(self.hidden2_u(f2))
    f2 = self.output2_u(f2)

    f3 = torch.selu(self.input3_u(u[:,2*self.noi:3*self.noi].reshape(-1,self.noi)))
    f3 = torch.selu(self.hidden3_u(f3))
    f3 = self.output3_u(f3)

    f4 = torch.selu(self.input4_u(u[:,3*self.noi:4*self.noi].reshape(-1,self.noi)))
    f4 = torch.selu(self.hidden4_u(f4))
    f4 = self.output4_u(f4)

    f = torch.cat((f1, f2, f3, f4), dim=1)

    # sigma
    s = sigma(t, self.k, self.n)

    # g
    g = torch.selu(self.input_t(t))
    g = torch.selu(self.hidden_t(g))
    g = self.output_t(g)

    return torch.sum(h*f*s*g + x0, dim=1).reshape(-1,1)

In [4]:
# Model error
def eval(model, testset):
    with torch.no_grad():
        pred_Y = model(testset.x0_data, testset.u_data, testset.t_data)

    r2  = r2score(pred_Y, testset.y_data)
    mse = msescore(pred_Y, testset.y_data)
    return r2.item(), mse.item()

In [5]:
# Data
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, n, H, noi):
    self.n   = n                                  # horizon length
    self.H   = H                                  # max window length
    self.noi = noi                                # number of input
    self.src_file = src_file                      # source file
    df = pd.read_csv(self.src_file, header=None)

    X0, U, T, Y = np.array([[1]], dtype=np.float32), np.ones((1, n*self.noi)), np.array([[1]], dtype=np.float32), np.array([[1]], dtype=np.float32)
    for i in range(df.shape[0]):
        row = np.array(df.iloc[i])
        for j in range(self.H - self.n):
            x0 = np.array([[row[self.H*self.noi + j]]])
            u  = np.array([row[j:j + self.n*self.noi]])
            for t in range(1, self.n + 1):
                y = np.array([[row[self.H*self.noi + j + t]]])
                t = np.array([[t]])

                X0 = np.concatenate((X0, x0))
                U  = np.concatenate((U, u))
                T  = np.concatenate((T, t))
                Y  = np.concatenate((Y, y))

    X0, U, T, Y = X0[1:], U[1:], T[1:], Y[1:]

    self.x0_data = torch.tensor(X0, dtype=torch.float32)
    self.u_data  = torch.tensor(U,  dtype=torch.float32)
    self.t_data  = torch.tensor(T,  dtype=torch.float32)
    self.y_data  = torch.tensor(Y,  dtype=torch.float32)

  def __len__(self):
    return len(self.x0_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    x0  = self.x0_data[idx]
    u   = self.u_data[idx]
    t   = self.t_data[idx]
    y   = self.y_data[idx]
    sample = {'x0':x0, 'u':u, 't':t, 'y':y}
    return sample

In [6]:
# Early stopping
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.0001*np.max(list[0: -patience])):
            return 1
    return 0

In [7]:
# Plot
def plot(net, dataset, size):
    with torch.no_grad():
        pred_Y = net(dataset.x0_data, dataset.u_data, dataset.t_data)

    plt.figure(figsize=size)
    plt.plot(dataset.y_data[0::4], 'b',   label=r'real',      linewidth=3)
    plt.plot(pred_Y[0::4],         'r--', label=r'predicted', linewidth=1)
    plt.ylabel(r'x(t)')
    plt.legend()
    plt.show()

In [8]:
# Train function
def train(net, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100):
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=train_ds.y_data.shape[0], shuffle=True)

    R2  = np.array([])
    MSE = np.array([])
    for epoch in range(0, max_epochs+1):
        net.train()
        loss  = 0
        count = 0
        for (_, batch) in enumerate(train_ldr):
            X0 = batch['x0']
            U  = batch['u']
            T  = batch['t']
            Y  = batch['y']

            optimizer.zero_grad()
            output = net(X0, U, T)             # compute the output of the Network
            loss_val = loss_func(output, Y)    # loss function
            loss += loss_val.item()            # accumulate
            loss_val.backward()                # gradients
            optimizer.step()                   # update paramters
            count += 1
        
        net.eval()
        R2  = np.append(R2, eval(net, test_ds)[0])
        MSE = np.append(MSE, eval(net, test_ds)[1])

        # if(epoch%500==0):
        #     print("epoch = %5d \t loss = %12.4f \t R2 = %12.4f \t MSE = %12.4f" % (epoch, loss/count, eval(net, test_ds)[0], eval(net, test_ds)[1]))
        
        if(early_stop(list = R2, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2, MSE

In [9]:
df_result = pd.DataFrame({'lr':[], 'p':[], 'b0':[], 'bi':[], 'trunk':[], 'best_epoch':[], 'R2':[], 'MSE':[]})

for _lr in [0.0001, 0.001, 0.01]:
    for _p in [4, 8, 12]:
        for b0 in [4, 8, 12]:
            for bi in [2, 4, 8]:
                for trunk in [4, 8, 12]:
                    # Hyperparameters
                    p   = _p          # size of branch and trunk ouput
                    n   = 4           # horizon window length
                    noi = 2           # number of inputs
                    H   = 512         # maximum window length

                    # Create Dataset and DataLoader objects
                    src_file_train = '0. Data/data_0.csv'
                    train_ds       = Data(src_file_train, n, H, noi)

                    src_file_test  = '0. Data/data_1.csv'
                    test_ds        = Data(src_file_test, n, H, noi)

                    # Create network
                    device = torch.device("cpu")
                    net = Net(n, p, noi, b0_size=b0, bi_size=bi, trunk_size=trunk).to(device)

                    # train model
                    lr         = _lr
                    min_epochs = 300
                    max_epochs = 100000
                    patience   = 200
                    R2, MSE = train(net, train_ds, test_ds, lr, min_epochs, max_epochs, patience)

                    # plot
                    # plot(net, train_ds, (20,2))
                    # plot(net, test_ds,  (20,2))

                    df_result.loc[len(df_result)] = [_lr, _p, b0, bi, trunk, np.argmax(R2), np.max(R2), np.min(MSE)]

                    # save
                    PATH = '2. Saved model/DeepONet_HVAC' + str(_lr) + str(_p) + str(b0) + str(bi) + str(trunk) + '.pt'
                    torch.save(net, PATH)

In [10]:
df_result.to_csv("stat.csv")

In [11]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 4,
                       ):
    print(df_result)

         lr     p    b0   bi  trunk  best_epoch          R2         MSE
0    0.0001   4.0   4.0  2.0    4.0     30404.0      0.9837      0.0519
1    0.0001   4.0   4.0  2.0    8.0     11142.0      0.9286      0.2273
2    0.0001   4.0   4.0  2.0   12.0     14454.0      0.5917      1.2998
3    0.0001   4.0   4.0  4.0    4.0     25145.0      0.9822      0.0568
4    0.0001   4.0   4.0  4.0    8.0     16808.0      0.6139      1.2292
5    0.0001   4.0   4.0  4.0   12.0     10501.0    -33.5438    109.9724
6    0.0001   4.0   4.0  8.0    4.0     10423.0      0.1527      2.6974
7    0.0001   4.0   4.0  8.0    8.0     22548.0      0.9807      0.0614
8    0.0001   4.0   4.0  8.0   12.0     17493.0      0.8419      0.5034
9    0.0001   4.0   8.0  2.0    4.0     12700.0      0.9807      0.0614
10   0.0001   4.0   8.0  2.0    8.0     15300.0      0.9717      0.0903
11   0.0001   4.0   8.0  2.0   12.0     19489.0      0.9032      0.3081
12   0.0001   4.0   8.0  4.0    4.0     37129.0      0.9812     